In [ ]:
# @title 🛠 Установка библиотек и импорты

!pip install -q crewai crewai_tools litellm

import os
import re
import shutil
import time
from google.colab import userdata, files
from getpass import getpass
from crewai import Agent, Task, Crew, Process
from crewai_tools import FileWriterTool

print("✅ Библиотеки установлены.")

# @title ⚙️ Настройка окружения

# 1. Фейковый ключ OpenAI
os.environ["OPENAI_API_KEY"] = "NA"

# 2. Настоящий ключ Groq
try:
    os.environ["GROQ_API_KEY"] = userdata.get('groq_api_key')
    print("✅ Groq API Key загружен из Secrets.")
except Exception:
    print("⚠️ Ключ не найден в Secrets.")
    os.environ["GROQ_API_KEY"] = getpass("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")

# 3. Выбор модели
MODEL_NAME = "groq/llama-3.1-8b-instant"

# 4. Подготовка рабочей папки
project_root = "/content/gen_project"
if os.path.exists(project_root):
    shutil.rmtree(project_root)
os.makedirs(project_root)

print(f"✅ Рабочая папка очищена: {project_root}")
print(f"✅ Используемая модель: {MODEL_NAME}")


✅ Библиотеки установлены.
⚠️ Ключ не найден в Secrets.
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··········
✅ Рабочая папка очищена: /content/gen_project
✅ Используемая модель: groq/llama-3.1-8b-instant


In [ ]:
%%writefile tasks.md
# ТЕХНИЧЕСКОЕ ЗАДАНИЕ: ENTERPRISE QUOTE APP

# ОБЩИЕ ПРАВИЛА ДЛЯ ВСЕХ АГЕНТОВ (ОБЯЗАТЕЛЬНО)

CONSOLE_PROMPT_TEMPLATE = """
Ты — агент {role}.

Выполни ТОЛЬКО инструкции блока #{block_idx} раздела "{section_name}".
Игнорируй все другие задачи и разделы.

Инструкции:
{instructions}

ОГРАНИЧЕНИЯ:
- НЕ используй File Writer Tool
- НЕ создавай файлы
- НЕ повторяй инструкции
- НЕ выходи за рамки блока #{block_idx}

ВАЖНО:
- Сначала выведи результат обычным текстом
- ПОСЛЕДНЕЙ строкой ответа ДОЛЖНО БЫТЬ строго:

Final Answer: DONE
"""

FILE_PROMPT_TEMPLATE = """
Ты — агент {role}.

Выполни ТОЛЬКО инструкции блока #{block_idx} раздела "{section_name}".
Игнорируй все другие задачи и разделы.

Инструкции:
{instructions}

ОГРАНИЧЕНИЯ:
- Используй ТОЛЬКО File Writer Tool
- directory всегда "{agent_dir}"
- НЕ создавай файлы, не указанные в инструкциях
- НЕ изменяй существующие файлы
- НЕ вызывай FileWriterTool после Final Answer

ВАЖНО:
После выполнения ОБЯЗАТЕЛЬНО верни ОДНУ строку:

Final Answer: DONE
"""



##  Глобальные настройки
- **Рабочая папка:** `./gen_project/`
- **Стек:** Java 21, Spring Boot 3, H2 Database (In-memory), Docker, Kotlin (для Android).
- **Инструментарий:** Агенты ОБЯЗАНЫ использовать `FileWriterTool` для сохранения файлов.

---

## СЕКЦИЯ 1: PRODUCT OWNER
**Задача:** Формирование требований

**Инструкция:**
1. Проанализируй идею приложения «Цитата дня».
2. Сформулируй **3 функциональные User Stories** в формате:
   - *Как [роль], я хочу [действие], чтобы [ценность]*.
3. Сохрани результат в файл `user_stories.txt` в рабочей папке.
4. После сохранения выведи `Final Answer: DONE`.

---

## СЕКЦИЯ 2: BACKEND DEVELOPER

Агент: Java Dev
Задача: Реализация backend API.

РАЗРЕШЕНО:
- создавать ТОЛЬКО перечисленные ниже файлы
- использовать ТОЛЬКО FileWriterTool

ЗАПРЕЩЕНО:
- изменять ранее созданные файлы
- повторно создавать файлы
- создавать сервисы, DTO, конфигурации (если не указано)

Инструкции:
1. Создай `pom.xml`
2. Создай `Quote.java` с полем `text` и аннотацией `@NotBlank`.
3. Создай `QuoteRepository.java` (интерфейс или заглушка).
4. Создай `QuoteController.java` с требованиями качества кода:Используй `@Slf4j` для логирования.Логируй входящие запросы и результат. Проверяй, что поле `text` не пустое. Оберни вызовы репозитория в try-catch или используй ResponseEntity для корректных HTTP-статусов (200, 404, 500).

Каждый файл:
- отдельный вызов FileWriterTool
- корректный Java-код, без шаблонов и заглушек

После пункта 4:
- НЕМЕДЛЕННО вывести `Final Answer: DONE`
- НЕ вызывать FileWriterTool
НЕ вызывать FileWriterTool

---

## СЕКЦИЯ 3: FRONTEND DEVELOPER
**Задача:** Веб-интерфейс.
**Инструкция:**
1. Создай файл `index.html`.
2. Внутри напиши HTML структуру и JS скрипт.
3. Скрипт должен делать `fetch('/api/quotes/random')` и отображать текст цитаты в `div`.
**ВАЖНО:** Сохрани файл в рабочую папку.

---

## СЕКЦИЯ 4: DEVOPS ENGINEER
**Задача:** Контейнеризация.
**Инструкция:**
1. Создай `Dockerfile`. Используй Multi-stage build (Maven image -> OpenJDK image).
2. Создай `docker-compose.yml` (service: app, port: 8080).
**ВАЖНО:** Сохрани оба файла в рабочую папку.

---

## СЕКЦИЯ 5: QA ENGINEER
**Задача:** Приемка.
**Инструкция:**
1. Создай файл `TEST_PLAN.md`.
2. Опиши 3 тест-кейса для проверки API.
**ВАЖНО:** Сохрани файл в рабочую папку.

---

## СЕКЦИЯ 6: MOBILE DEVELOPER
**Задача:** Создать Android клиент.
**Инструкция:**
1. Создай файл `MainActivity.kt`.
2. Используй Kotlin и Retrofit.
3. URL API для эмулятора Android: `http://10.0.2.2:8080/api/quotes/random`.
4. Сохрани файл в рабочую папку.

---

## СЕКЦИЯ 7: DOCUMENTATION
**Задача:** Написать документацию.
**Инструкция:**
1. Создай файл `README.md`.
2. Опиши стек технологий (включая Java, Kotlin, Docker).
3. Напиши инструкцию, как запустить проект через `docker-compose up`.


Overwriting tasks.md


In [ ]:
# @title 👥 Создание агентов (финальная версия)

file_writer = FileWriterTool()

config = {
    "allow_delegation": False,
    "verbose": True,
    "llm": MODEL_NAME
}

# Produ
product_owner = Agent(
    role="Product Owner",
    goal="Определение требований",
    backstory="PM / Business Analyst",
    tools=[file_writer],
    **config
)

# Агенты, создающие файлы
backend_dev = Agent(
    role="Java Dev",
    goal="Backend API",
    backstory="Java Expert. Always saves files.",
    tools=[file_writer],
    **config
)

frontend_dev = Agent(
    role="Frontend Developer",
    goal="Web UI",
    backstory="JS Developer. Always saves files.",
    tools=[file_writer],
    **config
)

devops_engineer = Agent(
    role="DevOps Engineer",
    goal="Containerization",
    backstory="DevOps Engineer. Always saves files.",
    tools=[file_writer],
    **config
)

qa_engineer = Agent(
    role="QA Engineer",
    goal="Testing",
    backstory="QA Specialist. Always writes thorough test cases and saves files.",
    tools=[file_writer],
    **config
)

android_dev = Agent(
    role="Android Developer",
    goal="Mobile App",
    backstory="Kotlin Expert. Always saves files.",
    tools=[file_writer],
    **config
)

tech_writer = Agent(
    role="Technical Writer",
    goal="Documentation",
    backstory="Documentation Expert. Always saves files.",
    tools=[file_writer],
    **config
)

print("✅ Команда укомплектована (7 агентов).")


✅ Команда укомплектована (7 агентов).


In [ ]:
!pip install 'litellm[proxy]'


In [ ]:
import os
import re
import time
import shutil
from litellm.exceptions import RateLimitError

completed_blocks = set()  # глобальный трекер завершённых блоков

def run_task_from_file(agent, section_name, mode="file", base_dir="./gen_project"):
    """Запуск заданий агента из tasks.md с корректным разбиением блоков"""

    with open("tasks.md", "r", encoding="utf-8") as f:
        full_specs = f.read()

    # --- извлекаем секцию ---
    lines = full_specs.splitlines()
    section_lines = []
    capture = False
    for line in lines:
        if re.match(r"##\s*" + re.escape(section_name), line):
            capture = True
            continue
        elif line.startswith("##") and capture:
            break
        if capture:
            section_lines.append(line.rstrip())

    # --- Формируем блоки ---
    blocks = []
    current_block = []
    for line in section_lines:
        line_strip = line.strip()
        if not line_strip:
            continue
        if re.match(r"^\d+\.", line_strip):
            if current_block:
                blocks.append(current_block)
            current_block = [line_strip]
        else:
            if current_block:
                current_block.append(line_strip)
    if current_block:
        blocks.append(current_block)

    # --- Создаём директорию агента ---
    agent_dir = os.path.join(base_dir, section_name.replace(" ", "_").lower())
    os.makedirs(agent_dir, exist_ok=True)

    # --- Для консольного режима сбрасываем папку и историю ---
    if mode == "console":
        if os.path.exists(agent_dir):
            shutil.rmtree(agent_dir)
        os.makedirs(agent_dir, exist_ok=True)
        completed_blocks.clear()

    # --- Выполнение блоков ---
    for idx, block in enumerate(blocks, start=1):
        block_id = f"{section_name}_{idx}"

        # Пропускаем блок, если уже выполнен
        if block_id in completed_blocks:
            print(f"[SKIP] {block_id} — уже выполнен")
            continue

        instructions = "\n".join(block)

        # универсальный prompt
        prompt = f"""
Ты — агент {agent.role}.

Выполни ТОЛЬКО инструкции блока #{idx} раздела '{section_name}'.

Инструкции:
{instructions}

ОГРАНИЧЕНИЯ:
- Используй ТОЛЬКО File Writer Tool
- directory всегда "{agent_dir}"
- НЕ создавай _instructions.md или служебные файлы

ВАЖНО:
После выполнения ОБЯЗАТЕЛЬНО верни:

Thought: I now know the final answer
Final Answer: DONE
"""

        # --- Консольный режим ---
        if mode == "console":
            print(f"\n[CONSOLE] {section_name} — блок {idx}\n{prompt}\n")
            completed_blocks.add(block_id)
            continue

        # --- File режим ---
        task = Task(description=prompt, expected_output="DONE", agent=agent)
        crew = Crew(
            agents=[agent],
            tasks=[task],
            process=Process.sequential,
            memory=False,
            planning=False,
            verbose=False
        )

        attempts = 0
        while attempts < 3:
            try:
                print(f"[RUN] {section_name} — блок {idx}")
                crew.kickoff()

                # проверка наличия файлов в директории
                files_in_block = os.listdir(agent_dir)
                if files_in_block:
                    completed_blocks.add(block_id)
                    print("[OK] Файлы созданы, блок завершён")
                    break

            except RateLimitError:
                print("[WAIT] RateLimit, 30 сек")
                time.sleep(30)
            except ValueError:
                files_in_block = os.listdir(agent_dir)
                if files_in_block:
                    completed_blocks.add(block_id)
                    print("[OK] Файлы есть, игнорируем пустой ответ")
                    break
                attempts += 1
                time.sleep(20)

        print("[WAIT] cooling 10 сек\n")
        time.sleep(10)

    print(f"[DONE] {section_name}\n")


In [ ]:
# Глобальный набор для отслеживания выполненных блоков
completed_blocks = set()
# Сброс истории выполнения
completed_blocks.clear()

# SECTION 1 — PRODUCT OWNER
run_task_from_file(
    agent=product_owner,
    section_name="СЕКЦИЯ 1: PRODUCT OWNER",
    mode="file"
)

# SECTION 2 — BACKEND
run_task_from_file(
    agent=backend_dev,
    section_name="СЕКЦИЯ 2: BACKEND DEVELOPER",
    mode="file"
)

# SECTION 3 — FRONTEND
run_task_from_file(
    agent=frontend_dev,
    section_name="СЕКЦИЯ 3: FRONTEND DEVELOPER",
    mode="file"
)

# SECTION 4 — DEVOPS
run_task_from_file(
    agent=devops_engineer,
    section_name="СЕКЦИЯ 4: DEVOPS ENGINEER",
    mode="file"
)

# SECTION 5 — QA
run_task_from_file(
    agent=qa_engineer,
    section_name="СЕКЦИЯ 5: QA ENGINEER",
    mode="file"
)

# SECTION 6 — MOBILE
run_task_from_file(
    agent=android_dev,
    section_name="СЕКЦИЯ 6: MOBILE DEVELOPER",
    mode="file"
)

# SECTION 7 — DOCUMENTATION
run_task_from_file(
    agent=tech_writer,
    section_name="СЕКЦИЯ 7: DOCUMENTATION",
    mode="file"
)


[RUN] СЕКЦИЯ 1: PRODUCT OWNER — блок 1


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Owner                                                                                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Ты — агент Product Owner.                                                                                      │
│                                                                                                                 │
│  Выполни ТОЛЬКО инструкции блока #1 раздела 'СЕКЦИЯ 1: PRODUCT OWNER'.                                          │
│                                                                                                                 │
│  Инструкции:                                                                                                    │
│  1. Проанализируй идею приложения «Цитата дня».                                                                 │
│                                                                                                                 │
│  ОГРАНИЧЕНИЯ:                                                                                                   │
│  - Используй ТОЛЬКО File Writer Tool                                                                            │
│  - directory всегда "./gen_project/секция_1:_product_owner"                                                     │
│  - НЕ создавай _instructions.md или служебные файлы                                                             │
│                                                                                                                 │
│  ВАЖНО:                                                                                                         │
│  После выполнения ОБЯЗАТЕЛЬНО верни:                                                                            │
│                                                                                                                 │
│  Thought: I now know the final answer                                                                           │
│  Final Answer: DONE                                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Owner                                                                                           │
│                                                                                                                 │
│  Thought: Thought: I should gather requirements for the 'Quote of the Day' application and write them down in   │
│  a file                                                                                                         │
│                                                                                                                 │
│  Using Tool: File Writer Tool                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "filename": "quote_of_the_day_requirements.md",                                                              │
│    "directory": "./gen_project/секция_1:_product_owner",                                                        │
│    "overwrite": true,                                                                                           │
│    "content": "Идея приложения \"Цитата дня\":\n- приложение предоставляет пользователям новую цитату каждый    │
│  день\n- цитата может быть выдернута из базы данных цитат или получить в API\n- пользователь может сохранить    │
│  цитату как избранную\n- пользователь может делиться цитатой через социальные сети"                             │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Content successfully written to ./gen_project/секция_1:_product_owner/quote_of_the_day_requirements.md         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Writer                                                                                        │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Ты — агент Technical Writer.                                                                                   │
│                                                                                                                 │
│  Выполни ТОЛЬКО инструкции блока #3 раздела 'СЕКЦИЯ 7: DOCUMENTATION'.                                          │
│                                                                                                                 │
│  Инструкции:                                                                                                    │
│  3. Напиши инструкцию, как запустить проект через `docker-compose up`.                                          │
│                                                                                                                 │
│  ОГРАНИЧЕНИЯ:                                                                                                   │
│  - Используй ТОЛЬКО File Writer Tool                                                                            │
│  - directory всегда "./gen_project/секция_7:_documentation"                                                     │
│  - НЕ создавай _instructions.md или служебные файлы                                                             │
│                                                                                                                 │
│  ВАЖНО:                                                                                                         │
│  После выполнения ОБЯЗАТЕЛЬНО верни:                                                                            │
│                                                                                                                 │
│  Thought: I now know the final answer                                                                           │
│  Final Answer: DONE                                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Writer                                                                                        │
│                                                                                                                 │
│  Thought: Thought: I need to write an instruction on how to run a project using `docker-compose up`, and I      │
│  should use the `File Writer Tool` to create a file that contains this instruction.                             │
│                                                                                                                 │
│  Using Tool: File Writer Tool                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "filename": "run_project_instruction.md",                                                                    │
│    "directory": "./gen_project/секция_7:_documentation",                                                        │
│    "overwrite": true,                                                                                           │
│    "content": "To run the project, navigate to the project directory and execute the following command:         │
│  `docker-compose up`. This will start all the containers and services necessary for the project to function     │
│  correctly."                                                                                                    │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Content successfully written to ./gen_project/секция_7:_documentation/run_project_instruction.md               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Received None or empty response from LLM call.
An unknown error occurred. Please check the details below.
Error details: Invalid response from LLM call - None or empty.
An unknown error occurred. Please check the details below.
Error details: Invalid response from LLM call - None or empty.
[OK] Файлы есть, игнорируем пустой ответ
[WAIT] cooling 10 сек

[DONE] СЕКЦИЯ 7: DOCUMENTATION



In [ ]:
# @title 📦 Архивация результата (gen_project)

import shutil
import os

PROJECT_DIR = "gen_project"
ARCHIVE_NAME = "enterprise_quote_app"

# Удаляем старый архив, если есть
if os.path.exists(f"{ARCHIVE_NAME}.zip"):
    os.remove(f"{ARCHIVE_NAME}.zip")

# Создаём архив
shutil.make_archive(
    ARCHIVE_NAME,
    "zip",
    PROJECT_DIR
)

print(f"[OK] Архив создан: {ARCHIVE_NAME}.zip")


[OK] Архив создан: enterprise_quote_app.zip
